<a href="https://colab.research.google.com/github/AkiraTerao/AT/blob/master/mie_scatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as pyllot
%matplotlib inline

function musgp = getMieScatter(lambda, dia, fv, npar,nmed)

% function musgp = getMieScatter(lambda, dia, fv)

%  fv            = volume fraction of spheres in medium (eg., fv = 0.05)

%  lambda        = wavelength in um (eg., lambda = 0.633)

%  dia           = sphere diameter in um (eg., dia_um = 0.0500)

%  npar          = particle refractive index (eg. polystyrene = 1.57)

%  nmed          = medium refractive index (eg., water = 1.33)

%                  Note: npar and nmed can be imaginary numbers.

%  returns musgp = [mus g musp]  

%       mus      = scattering coefficient [cm^-1]

%       g        = anisotropy of scattering [dimensionless]

%       musp     = reduced scattering coefficient [cm^-1]

%  Uses

%       Mie.m, which uses mie_abcd.m, from Maetzler 2002

In [2]:
fv=0.05
lam=0.633 # um
dia=0.05 # um
npar=1.57 
nmed=1.33

In [3]:
Vsphere = 4/3*np.pi*(dia/2)**3;     # volume of sphere
rho     = fv/Vsphere;           # #/um^3, concentration of spheres
m = npar/nmed;                  # ratio of refractive indices
x = np.pi*dia/(lam/nmed);       # ratio circumference/wavelength in medium

In [4]:
if x==0: # To avoid a singularity at x=0
  result=np.array([np.real(m),np.imag(m), 0, 0, 0, 0, 0, 0, 1.5])
elif x>0: # This is the normal situation
  nmax=round(2+x+4*x**(1/3));

In [5]:
n1=nmax-1;
n=np.arange(1,nmax+1);cn=2*n+1; c1n=n*(n+2)/(n+1); c2n=cn/n/(n+1);
x2=x*x;

mie_abcd

In [6]:
nmax=round(2+x+4*x**(1/3));
n=np.arange(1,nmax+1); nu =(n+0.5); z=m*x; m2=m*m;
sqx= np.sqrt(0.5*np.pi/x); sqz= np.sqrt(0.5*np.pi/z);

In [7]:
from scipy.special import jv, yv
bx = jv(nu, x)*sqx
bz = jv(nu, z)*sqz
yx = yv(nu, x)*sqx
hx = bx+1j*yx;

In [8]:
b1x=np.concatenate([[np.sin(x)/x],bx[0:nmax-1]])
b1z=np.concatenate([[np.sin(z)/z],bz[0:nmax-1]])
y1x=np.concatenate([[-np.cos(x)/x],yx[0:nmax-1]])
h1x= b1x+1j*y1x;

In [9]:
ax = x*b1x-n*bx;
az = z*b1z-n*bz;
ahx= x*h1x-n*hx;

In [13]:
an = (m2*bz*ax-bx*az)/(m2*bz*ahx-hx*az);
bn = (bz*ax-bx*az)/(bz*ahx-hx*az);
cn = (bx*ahx-hx*ax)/(bz*ahx-hx*az);
dn = m*(bx*ahx-hx*ax)/(m2*bz*ahx-hx*az);
result=np.array([an, bn, cn, dn]);
f=result

end of mie_abcd

In [14]:
anp=np.real(f[0,:]); anpp=np.imag(f[0,:]);
bnp=np.real(f[1,:]); bnpp=np.imag(f[1,:]);

In [20]:
g1[0,:n1]=anp[1:nmax];

In [21]:
g1=np.zeros([4,nmax]); g1[0,:n1]=anp[1:nmax]; g1[1,:n1]=anpp[1:nmax]; g1[2,:n1]=bnp[1:nmax]; g1[3,:n1]=bnpp[1:nmax]

In [22]:
dn=cn*(anp+bnp);
q=sum(dn);
qext=2*q/x2;
en=cn*(anp*anp+anpp*anpp+bnp*bnp+bnpp*bnpp);
q=sum(en);
qsca=2*q/x2;
qabs=qext-qsca;
fn=(f[0,:]-f[1,:])*cn;
gn=(-1)**n;
f[2,:]=fn*gn;
q=np.sum(f[2,:]);

(-5.296789323131095e-06-2.9963865058671924e-08j)

In [23]:
qb=q*q/x2;
asy1=c1n*(anp*g1[0,:]+anpp*g1[1,:]+bnp*g1[2,:]+bnpp*g1[3,:]);
asy2=c2n*(anp*bnp+anpp*bnpp);
asy=4/x2*np.sum(asy1+asy2)/qsca;
qratio=qb/qsca;
result=np.array([np.real(m), np.imag(m), x, qext, qsca, qabs, qb, asy, qratio]);

In [25]:
u=result
qsca = u[4];                    # scattering efficiency, Qsca
g    = u[7];                    # anisotropy, g

A       = np.pi*dia**2/4;          # geometrical cross-sectional area, um^2
sigma_s = qsca*A;               # scattering cross-section, um^2
mus     = sigma_s*rho*1e4;      # scattering coeff. cm^-1
musp    = mus*(1-g);            # reduced scattering coeff. cm^-1


In [33]:
print('----- choice:')
print(f'lambda = {lam: .4f} um')
print(f'diameter = {dia: .4f} um')
print(f'rho =  {rho: .4f} 1/um^3')
print(f'npar = {npar: .3f}')
print(f'nmed = {nmed: .3f}')

----- choice:
lambda =  0.6330 um
diameter =  0.0500 um
rho =   763.9437 1/um^3
npar =  1.570
nmed =  1.330


In [29]:
number = 0.45
print('{0:.4f} is {0:.2%}'.format(number))
# 0.4500 is 45.00%

print('{number:.4f} is {number:.2%}')

0.4500 is 45.00%
{number:.4f} is {number:.2%}


In [30]:
print(f'lambda = {lam: .4f} um')

lambda =  0.6330 um
